In [93]:
# import packages
import pandas as pd
import numpy as np
import sklearn
import sklearn.neighbors
import sklearn.metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from collections import defaultdict
import seaborn as sns
import numpy as np

#lemmatization libraries 
import nltk.stem
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# punctuation + stop words libraries:
from nltk.corpus import stopwords
import re, string, timeit

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ophir\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ophir\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ophir\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1(a) Parsing the txt files:

## Question 1(a): Parsing Yelp, IMDB, and Amazon database:

#### Parsing Yelp

In [94]:
df_yelp = pd.read_csv("yelp_labelled.txt", sep="\t", header=None, names=["sentence", "label"])

How many values are there? How many positive and negative emotions (0/1)?

In [95]:
df_yelp.label.value_counts()

1    500
0    500
Name: label, dtype: int64

#### Parsing IMDB

In [96]:
df_imdb = pd.read_csv("imdb_labelled.txt", sep=" \t", header=None, names=["sentence", "label"])
#Notice: IMDB required another space in order to parse the 1000 lines because it was badly formatted

<ipython-input-96-0a2494051e90>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_imdb = pd.read_csv("imdb_labelled.txt", sep=" \t", header=None, names=["sentence", "label"])


How many values are there? How many positive and negative emotions (0/1)?

In [97]:
df_imdb.label.value_counts()

1    500
0    500
Name: label, dtype: int64

#### Parsing Amazon 

In [98]:
df_amazon = pd.read_csv("amazon_cells_labelled.txt", sep="\t", header=None, names=["sentence", "label"])

How many values are there? How many positive and negative emotions (0/1)?

In [99]:
df_amazon.label.value_counts()

1    500
0    500
Name: label, dtype: int64

#### Results - Amazon, IMDB and Yelp are balanced (50% negative 50% positive)
    

## Question 2(b): Preprocessing

Pick your preprocessing strategy. Since these sentences are online reviews, they may contain significant amounts of noise and garbage. You may or may not want to do one or all of
the following.

• Lowercase all of the words.

• Lemmatization of all the words (i.e., convert every word to its root so that all of “running,”
“run,” and “runs” are converted to “run” and and all of “good,” “well,” “better,” and “best”
are converted to “good”; this is easily done using nltk.stem).

• Strip punctuation.

• Strip the stop words, e.g., “the”, “and”, “or”.

• Something else? Tell us about it.

#### Step 1: Lowercase all databases

In [100]:
df_yelp["sentence"] = df_yelp["sentence"].str.lower()

In [101]:
df_imdb["sentence"] = df_imdb["sentence"].str.lower()

In [102]:
df_amazon["sentence"] = df_amazon["sentence"].str.lower()

#### Step 2: Punctuation Strip

In [103]:
#function that removes punctuation:
def remove_punctuation(sentence):
    sentence = re.sub(r'[^\w\s]','',sentence)
    return(sentence)

In [104]:
df_yelp["sentence"] = df_yelp['sentence'].apply(remove_punctuation)

In [105]:
df_imdb["sentence"] = df_imdb['sentence'].apply(remove_punctuation)

In [106]:
df_amazon["sentence"] = df_amazon['sentence'].apply(remove_punctuation)

#### Step 3: Strip the stop words, e.g., “the”, “and”, “or”

In [107]:
stop = stopwords.words('english')

In [108]:
df_yelp['sentence'] = df_yelp['sentence'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [109]:
# make sure there are no stop words in the database:
# df_yelp["sentence"].str.contains("the", na=False)

In [110]:
df_imdb['sentence'] = df_imdb['sentence'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [111]:
df_amazon['sentence'] = df_amazon['sentence'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

#### Step 4: Lemmatiation of words

Defining:

In [112]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
wnl = WordNetLemmatizer()

Define a function that returns a lemmatized sentence:

In [113]:
def lemmatize_text(text):
    list2 = nltk.word_tokenize(text)
    lemmatized_sentence = ' '.join([wnl.lemmatize(words) for words in list2])
    return(lemmatized_sentence)

Apply on the "sentence" column on all datasets:

In [114]:
df_yelp['sentence'] = df_yelp['sentence'].apply(lambda x: lemmatize_text(x))

In [115]:
df_imdb['sentence'] = df_imdb['sentence'].apply(lambda x: lemmatize_text(x))

In [116]:
df_amazon['sentence'] = df_amazon['sentence'].apply(lambda x: lemmatize_text(x))

## 2(C): Split training and test data

For each file use the first 400 instances for each label as the training set and the remaining 100 instances as testing set.

In total, there are 2400 reviews for training and 600 reviews for testing.

#### Yelp - Divide to positive and negative data set and extract from there the train / test datasets:

In [117]:
# Divide into positive / negative datasets:
df_yelp_pos = df_yelp[df_yelp['label'] == 1].reset_index(drop=True)   # positive emotion = 1
df_yelp_neg = df_yelp[df_yelp['label'] == 0].reset_index(drop=True)   # Negative emotion = 0

# For the positive dataset - divide into train / test datasets:
df_yelp_train_pos = df_yelp_pos[:400]
df_yelp_test_pos = df_yelp_pos[400:500]

# For the negative dataset - divide into train / test datasets:
df_yelp_train_neg = df_yelp_neg[:400]
df_yelp_test_neg = df_yelp_neg[400:500]

#### IMDB - Divide to positive and negative data set and extract from there the train / test datasets:

In [118]:
# Divide into positive / negative datasets:
df_imdb_pos = df_imdb[df_imdb['label'] == 1].reset_index(drop=True)   # positive emotion = 1
df_imdb_neg = df_imdb[df_imdb['label'] == 0].reset_index(drop=True)   # Negative emotion = 0

# For positive - train / test datasets:
df_imdb_train_pos = df_imdb_pos[:400]
df_imdb_test_pos = df_imdb_pos[400:500]

# For the negative dataset - divide into train / test datasets:
df_imdb_train_neg = df_imdb_neg[:400]
df_imdb_test_neg = df_imdb_neg[400:500]

#### Amazon - Divide to positive and negative data set and extract from there the train / test datasets:

In [119]:
# Divide into positive / negative datasets:
df_amazon_pos = df_amazon[df_amazon['label'] == 1].reset_index(drop=True)   # positive emotion = 1
df_amazon_neg = df_amazon[df_amazon['label'] == 0].reset_index(drop=True)   # Negative emotion = 0

# For positive - train / test datasets:
df_amazon_train_pos = df_amazon_pos[:400]
df_amazon_test_pos = df_amazon_pos[400:500]

# For the negative dataset - divide into train / test datasets:
df_amazon_train_neg = df_amazon_neg[:400]
df_amazon_test_neg = df_amazon_neg[400:500]

### Concat - train_df & test_df:

In [120]:
df_train = pd.concat([df_amazon_train_neg, df_amazon_train_pos, df_yelp_train_neg, df_yelp_train_pos,
                                   df_imdb_train_neg, df_imdb_train_pos], ignore_index=True)

#self check
len(df_train)

2400

In [121]:
df_test = pd.concat([df_amazon_test_neg, df_amazon_test_pos, df_yelp_test_neg, df_yelp_test_pos,
                                   df_yelp_test_neg, df_yelp_test_pos], ignore_index=True)

#self check
len(df_test)

600

## Question 2(d)

Bag of Words model.

Extract features and then represent each review using bag of words model, i.e., every word in the review becomes its own element in a feature vector. 
In order to do this, first, make one pass through all the reviews in the training set (Explain why we can’t
use testing set at this point) and build a dictionary of unique words. 
Then, make another pass through the review in both the training set and testing set and count up the occurrences of
each word in your dictionary. 
The i-th element of a review’s feature vector is the number of occurrences of the ith dictionary word in the review. 
Implement the bag of words model and report feature vectors of any two reviews in the training set. 


#### Create a dictionary with all unique words:

In [122]:
# Dictionary with all unique words and their count:
word_freq = defaultdict(int)
for row in range(0,len(df_train)):
    for i in df_train['sentence'][row].split():
        word_freq[i] += 1

#print(word_freq)

In [123]:
# Put 0-s instead of counting every unique word:
feature_vector = {x:0 for x in word_freq}

#print(feature_vector)

In [124]:
def feature_vector_func(sentence):
    word_list = sentence.split()
    counter = 0
    for word in word_freq:
        if word in word_list:
            feature_vector[word] = counter + 1
        else:
            feature_vector[word] = counter
    return(feature_vector)

#### Applying the function to all rows in df_train to get all feature vectors:

In [125]:
df_train['feature_vector_dict'] = df_train['sentence'].apply(lambda x: feature_vector_func(x))

In [126]:
df_train.head()

,sentence,label,feature_vector_dict
0,way plug u unless go converter,0,"{'way': 0, 'plug': 0, 'u': 0, 'unless': 0, 'go..."
1,tied charger conversation lasting 45 minutesma...,0,"{'way': 0, 'plug': 0, 'u': 0, 'unless': 0, 'go..."
2,jiggle plug get line right get decent volume,0,"{'way': 0, 'plug': 0, 'u': 0, 'unless': 0, 'go..."
3,several dozen several hundred contact imagine ...,0,"{'way': 0, 'plug': 0, 'u': 0, 'unless': 0, 'go..."
4,needle say wasted money,0,"{'way': 0, 'plug': 0, 'u': 0, 'unless': 0, 'go..."


#### Applying the function to all rows in the same manner for df_test:

In [127]:
df_test['feature_vector_dict'] = df_test['sentence'].apply(lambda x: feature_vector_func(x))

In [128]:
df_test.head()

,sentence,label,feature_vector_dict
0,doesnt work,0,"{'way': 0, 'plug': 0, 'u': 0, 'unless': 0, 'go..."
1,unfortunately work,0,"{'way': 0, 'plug': 0, 'u': 0, 'unless': 0, 'go..."
2,three broke within two month use,0,"{'way': 0, 'plug': 0, 'u': 0, 'unless': 0, 'go..."
3,dont buy,0,"{'way': 0, 'plug': 0, 'u': 0, 'unless': 0, 'go..."
4,father v265 battery dying,0,"{'way': 0, 'plug': 0, 'u': 0, 'unless': 0, 'go..."


#### Compare any two feature vectors of any two reviews in the training set:

In [129]:
df_train['feature_vector_dict'][:2]

0    {'way': 0, 'plug': 0, 'u': 0, 'unless': 0, 'go...
1    {'way': 0, 'plug': 0, 'u': 0, 'unless': 0, 'go...
Name: feature_vector_dict, dtype: object

## Question 2(e) - Pick your postprocessing strategy

#### We chose to use the 4th method - standardize the data by subtracting the mean and dividing by the variance.

Standardization, on the other hand, can be helpful in cases where the data follows a Gaussian distribution. 

However, this does not have to be necessarily true. Also, unlike normalization, standardization does not have a bounding range. 

So, even if you have outliers in your data, they will not be affected by standardization.

In [130]:
#Function that converts the dict "feature_vector" into an array:
def dict_to_array(vector):
    data = list(vector.values())
    an_array = np.array(data)
    return(an_array)

In [131]:
# data = list(df_train['feature_vector'][1].values())
# an_array = np.array(data)
# print(an_array)

In [132]:
df_train['feature_vector_array'] = df_train['feature_vector_dict'].apply(lambda x: dict_to_array(x))

In [133]:
#self check - ignore
df_train['feature_vector_array'][1]

array([0, 0, 0, ..., 0, 0, 0])

In [134]:
#self check - ignore
for i in df_train['feature_vector_array'][1]:
    if i == 1:
        print(i)

1
1
1
1
1


In [135]:
#insert: df_train['array_vector'][i], get the standardized array:
def standard(dataset):
    standardized_array = (dataset - np.average(dataset)) / (np.std(dataset))
    return(standardized_array)

In [136]:
df_train['standard_array'] = df_train['feature_vector_array'].apply(lambda x: standard(x))

In [137]:
#self check - ignore
df_train['standard_array'][1]

array([-0.03431587, -0.03431587, -0.03431587, ..., -0.03431587,
       -0.03431587, -0.03431587])

In [138]:
#self check 
# for i in df_train['standard_array'][1]:
#     if i != -0.04606127153588057:
#         print(i, df_train.index)

In [139]:
df_train.head()

,sentence,label,feature_vector_dict,feature_vector_array,standard_array
0,way plug u unless go converter,0,"{'way': 0, 'plug': 0, 'u': 0, 'unless': 0, 'go...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.034315869474430515, -0.034315869474430515,..."
1,tied charger conversation lasting 45 minutesma...,0,"{'way': 0, 'plug': 0, 'u': 0, 'unless': 0, 'go...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.034315869474430515, -0.034315869474430515,..."
2,jiggle plug get line right get decent volume,0,"{'way': 0, 'plug': 0, 'u': 0, 'unless': 0, 'go...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.034315869474430515, -0.034315869474430515,..."
3,several dozen several hundred contact imagine ...,0,"{'way': 0, 'plug': 0, 'u': 0, 'unless': 0, 'go...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.034315869474430515, -0.034315869474430515,..."
4,needle say wasted money,0,"{'way': 0, 'plug': 0, 'u': 0, 'unless': 0, 'go...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.034315869474430515, -0.034315869474430515,..."


### For the test data:

In [140]:
df_test['feature_vector_array'] = df_test['feature_vector_dict'].apply(lambda x: dict_to_array(x))
df_test['standard_array'] = df_test['feature_vector_array'].apply(lambda x: standard(x))

In [141]:
len(df_test['standard_array'][1])

4251

## Question 2(f) - Sentiment Prediction

Train a naive Bayes model on the training set and test on the testing set. 

Report the classification accuracy and confusion matrix.

In [227]:
train_X = df_train["standard_array"]
test_X = df_test["standard_array"]

In [228]:
train_X = pd.DataFrame(train_X.to_list())
test_X = pd.DataFrame(test_X.to_list())

In [229]:
train_X = train_X.to_numpy()
test_X = test_X.to_numpy()

In [230]:
train_Y = df_train['label']
test_Y = df_test['label']

In [231]:
train_X

array([[-0.03431587, -0.03431587, -0.03431587, ..., -0.03431587,
        -0.03431587, -0.03431587],
       [-0.03431587, -0.03431587, -0.03431587, ..., -0.03431587,
        -0.03431587, -0.03431587],
       [-0.03431587, -0.03431587, -0.03431587, ..., -0.03431587,
        -0.03431587, -0.03431587],
       ...,
       [-0.03431587, -0.03431587, -0.03431587, ..., -0.03431587,
        -0.03431587, -0.03431587],
       [-0.03431587, -0.03431587, -0.03431587, ..., -0.03431587,
        -0.03431587, -0.03431587],
       [-0.03431587, -0.03431587, -0.03431587, ..., -0.03431587,
        -0.03431587, -0.03431587]])

In [232]:
n = train_X.shape[0] #number of rows
d = train_X.shape[1] #number of unique words = features in feature vector
K = 2 #number of classes - label 1 or label 0

In [256]:
psis = np.zeros([K,d])
phis = np.zeros([K])

In [257]:
for k in range(K):
    X_k = train_X[train_Y == k]
    phis[k] = X_k.shape[0] / float(n)
    psis[k] = np.mean(X_k, axis=0) #build a function with the mean for label 0 and mean for label 0 as an array

In [239]:
# print(psis.shape, phis.shape, train_X.shape)

In [291]:
# we can implement this in numpy
def nb_predictions(x, psis, phis):
    """This returns class assignments and scores under the NB model.
    
    We compute \arg\max_y p(y|x) as \arg\max_y p(x|y)p(y)
    """
    # adjust shapes
    n , d = x.shape
    x = np.reshape(x, (1,n,d))
    psis = np.reshape(psis, (K, 1, d))
    
    # clip probabilities to avoid log(0)
    psis = psis.clip(1e-14, 1-1e-14) #understand
    
    # compute log-probabilities
    logpy = np.log(phis).reshape(K,1)
    logpxy = x * np.log(psis) + (1-x) * np.log(1-psis)
    logpyx = logpxy.sum(axis=2) + logpy

    return logpyx.argmax(axis=0).flatten(), logpyx.reshape([K,n])

### Naive Bayes on the Training Set

In [279]:
idx, logpyx = nb_predictions(train_X, psis, phis)

In [281]:
(idx == train_Y).mean()

0.5

In [288]:
idx #len(idx)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

### Naive Bayes on the Data Set

In [283]:
idx_test, logpyx_test = nb_predictions(test_X, psis, phis)

In [289]:
idx_test #len(idx_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,